In [ ]:
using LinearAlgebra
include("../src/gf.jl")

In [ ]:
using OrdinaryDiffEq, Parameters, MuladdMacro
include("../src/common.jl")

### Define problem

In [ ]:
tspan = (0.0, 1.0)
dt = 0.1

steps = ceil(Int, (tspan[2] - tspan[1])/dt) + 1;

In [ ]:
# gf = LesserGF(zeros(Float64, 1, 1));
data = reshape([ones(2,2),zeros(2,2),zeros(2,2),zeros(2,2)], 2, 2)
gf = LesserGF(data);

In [ ]:
function f(u, p, t, t′)
    [u[1][t,t′]]
end

In [ ]:
u = [data, ];

In [ ]:
prob = ODEProblem(f, u, tspan);

### Kadanoff-Baym timestepper tests

In [ ]:
alg = KB{ABM43}()

@show OrdinaryDiffEq.alg_order(alg)
@show OrdinaryDiffEq.isfsal(alg);

In [ ]:
# DiffEqBase.__init(prob, alg, dt)

In [ ]:
integrator = KBIntegrator((1,1), (0,1), 1, prob.u0, prob.f, prob.f(u,nothing,1,1), nothing, nothing, nothing, false)

In [ ]:
cache = OrdinaryDiffEq.HeunConstantCache()
eulerHeun!(integrator, cache)
prob.u0

In [ ]:
cache = OrdinaryDiffEq.ABM43ConstantCache(zero.(integrator.fsalfirst),zero.(integrator.fsalfirst),zero.(integrator.fsalfirst),1)
abm43!(integrator, cache)
prob.u0

In [ ]:
function test_callback_outofplace(alg; kwargs...)
    f = (u, p, t) -> copy(u)
    cb = ContinuousCallback((u,t,int) -> u[1] - exp(1), terminate!)
    prob = ODEProblem(f, [1.0], (0.0, 2.0), callback=cb)
    sol = solve(prob, alg; kwargs...)
    sol.u[end][1] ≈ exp(1)
end

@test test_callback_outofplace(KadanoffBaym(); dt=0.1)